In [1]:
!pip install gradio gputil pandas transformers

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 62.4 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 81.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.9 MB/s eta 0:00:00
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=43bc78b3174d4f0f4b8980a30d41cf2cc3265cb4a7a3fb644c1df6d794bc7ca5
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=c7b0c06c7e78fa8b468d567467cf87f7674f166d5b1a482c98f57cec8311cb21
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a00

In [ ]:
def evaluate_response(problem):
    start_time = time.time()
    
    problem = problem + '\nPlease reason step by step, and put your final answer within \\boxed{}.'
    messages = [
        {"role": "user", "content": problem}
    ]
    
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    attention_mask = (input_tensor != tokenizer.pad_token_id).long()
    
    outputs = model.generate(
        input_tensor.to(model.device), 
        attention_mask=attention_mask.to(model.device), 
        max_new_tokens=100
    )

    result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for evaluation: {elapsed_time:.2f} seconds")
    
    return result

In [2]:
import gradio as gr

import torch
import pandas as pd
import GPUtil
# client = InferenceClient("HuggingFaceH4/zephyr-7b-beta")
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, set_seed
# from accelerate import infer_auto_device_map as iadm
 
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
# Check GPU usage
gpus = GPUtil.getGPUs()
for gpu in gpus:
    print(f"GPU ID: {gpu.id}")
    print(f"Name: {gpu.name}")
    print(f"Load: {gpu.load * 100:.1f}%")
    print(f"Memory Free: {gpu.memoryFree:.1f}MB")
    print(f"Memory Used: {gpu.memoryUsed:.1f}MB")
    print(f"Memory Total: {gpu.memoryTotal:.1f}MB")
    print(f"Temperature: {gpu.temperature} °C")
    print(f"UUID: {gpu.uuid}\n")

# Load and check the file (update the path as needed)
file_path = '/kaggle/input/deepmindmathdata/deepmind_mathemathics.csv'  # replace with your file path
df = pd.read_csv(file_path)

# General parameters of the file
print("First few rows of the file:")
print(df.head())

print("\nSummary statistics of the file:")
print(df.describe())

print("\nInfo about the file:")
print(df.info())

print("\nShape of the file (rows, columns):")
print(df.shape)
 

"""
For more information on `huggingface_hub` Inference API support, please check the docs: https://huggingface.co/docs/huggingface_hub/v0.22.2/en/guides/inference
"""


model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id




def evaluate_response(problem):
#     problem=b'what is angle x if angle y is 60 degree and angle z in 60 degree of a traingle'
    problem=problem+'\nPlease reason step by step, and put your final answer within \\boxed{}.'
    messages = [
        {"role": "user", "content": problem}
    ]
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    attention_mask = (input_tensor != tokenizer.pad_token_id).long()
    outputs = model.generate(
        input_tensor.to(model.device), 
        attention_mask=attention_mask.to(model.device), 
        max_new_tokens=100
    )
    result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
     
#     result_output, code_output = process_output(raw_output)
    return result
 
demo = gr.Interface(
        fn=evaluate_response,
        inputs=[gr.Textbox(label="Question")],
        outputs=gr.Textbox(label="Answer"),
        title="Question and Answer Interface",
        description="Enter a question."
    )

  
if __name__ == "__main__":
    demo.launch()

GPU ID: 0
Name: Tesla T4
Load: 0.0%
Memory Free: 15095.0MB
Memory Used: 1.0MB
Memory Total: 15360.0MB
Temperature: 43.0 °C
UUID: GPU-d29f2916-dc36-d332-d69b-b22208eac439

GPU ID: 1
Name: Tesla T4
Load: 0.0%
Memory Free: 15095.0MB
Memory Used: 1.0MB
Memory Total: 15360.0MB
Temperature: 42.0 °C
UUID: GPU-8e4fc71a-4385-6c22-9d4d-9153afe893a2

First few rows of the file:
                                            question  \
0  b'Let w(f) be the second derivative of -18*f +...   
1  b'Suppose 0*l + 3*l - 5*u - 16 = 0, 3*u + 15 =...   
2  b'Determine t so that -10*t - 5*t + 48*t + 5*t...   
3  b'Let c = 3272/5 - 652. Let l(z) = 2*z - 14. L...   
4  b'Let y = 26 - 12. Suppose 8*t + 14 - y = 0. S...   

                            answer  
0                      b'c**5/2\n'  
1                b'2*(i + 1)**3\n'  
2                      b'-13, 0\n'  
3  b'4*v**3*(v + 2)*(3*v + 2)/5\n'  
4                     b'0, 1, 2\n'  

Summary statistics of the file:
                                  ques

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://d6975f7e975b7812f7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [3]:
evaluate_response("2+2")

AttributeError: 'bool' object has no attribute 'long'

In [ ]:
!pip install psutil

In [3]:
import gradio as gr
import torch
import pandas as pd
import GPUtil
import psutil
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

#check GPU availability and usage
if torch.cuda.is_available():
    print("CUDA is available. Checking GPU usage...")
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU ID: {gpu.id}")
        print(f"Name: {gpu.name}")
        print(f"Load: {gpu.load * 100:.1f}%")
        print(f"Memory Free: {gpu.memoryFree:.1f}MB")
        print(f"Memory Used: {gpu.memoryUsed:.1f}MB")
        print(f"Memory Total: {gpu.memoryTotal:.1f}MB")
        print(f"Temperature: {gpu.temperature} °C")
        print(f"UUID: {gpu.uuid}\n")
else:
    print("CUDA is not available. Checking CPU usage...")
    # CPU usage information
    print(f"CPU usage: {psutil.cpu_percent()}%")
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Load and check the file (update the path as needed)
file_path = '/kaggle/input/deepmindmathdata/deepmind_mathemathics.csv'  # replace with your file path
df = pd.read_csv(file_path)

# General parameters of the file
print("First few rows of the file:")
print(df.head())

print("\nSummary statistics of the file:")
print(df.describe())

print("\nInfo about the file:")
print(df.info())

print("\nShape of the file (rows, columns):")
print(df.shape)

# Model setup
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

# Function to evaluate response
def evaluate_response(problem):
    start_time = time.time()
    
    problem = problem + '\nPlease reason step by step, and put your final answer within \\boxed{}.'
    messages = [
        {"role": "user", "content": problem}
    ]
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=100)

    result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for evaluation: {elapsed_time:.2f} seconds")
    
    return result

# Gradio interface
demo = gr.Interface(
    fn=evaluate_response,
    inputs=[gr.Textbox(label="Question")],
    outputs=gr.Textbox(label="Answer"),
    title="Question and Answer Interface",
    description="Enter a question."
)

if __name__ == "__main__":
    demo.launch()


CUDA is not available. Checking CPU usage...
CPU usage: 21.1%
Memory usage: 4.2%
First few rows of the file:
                                            question  \
0  b'Let w(f) be the second derivative of -18*f +...   
1  b'Suppose 0*l + 3*l - 5*u - 16 = 0, 3*u + 15 =...   
2  b'Determine t so that -10*t - 5*t + 48*t + 5*t...   
3  b'Let c = 3272/5 - 652. Let l(z) = 2*z - 14. L...   
4  b'Let y = 26 - 12. Suppose 8*t + 14 - y = 0. S...   

                            answer  
0                      b'c**5/2\n'  
1                b'2*(i + 1)**3\n'  
2                      b'-13, 0\n'  
3  b'4*v**3*(v + 2)*(3*v + 2)/5\n'  
4                     b'0, 1, 2\n'  

Summary statistics of the file:
                                  question         answer
count                              1999998        1999998
unique                             1999737         701501
top     b'Factor -2/7 + 0*y + 2/7*y**2.\n'  b'-1, 0, 1\n'
freq                                     3          51777

Info abo

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://eb66d3dc4c846d9860.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [2]:
!pip install gradio gputil pandas transformers psutil

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 75.3 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 92.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.8 MB/s eta 0:00:00
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=d8816c7802efe5017e4d97e32184970d9a757ca9164954c012723ab6445979ec
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=bb26f128d6cdff502583d6d0f71dcf08b1adaf1982c35e5063ed3624ea4153a3
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a002

In [4]:


import gradio as gr
import torch
import pandas as pd
import GPUtil
import psutil
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

# Check GPU availability and usage
if torch.cuda.is_available():
    print("CUDA is available. Checking GPU usage...")
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU ID: {gpu.id}")
        print(f"Name: {gpu.name}")
        print(f"Load: {gpu.load * 100:.1f}%")
        print(f"Memory Free: {gpu.memoryFree:.1f}MB")
        print(f"Memory Used: {gpu.memoryUsed:.1f}MB")
        print(f"Memory Total: {gpu.memoryTotal:.1f}MB")
        print(f"Temperature: {gpu.temperature} °C")
        print(f"UUID: {gpu.uuid}\n")
else:
    print("CUDA is not available. Checking CPU usage...")
    # CPU usage information
    print(f"CPU usage: {psutil.cpu_percent()}%")
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Load and check the file (update the path as needed)
file_path = '/kaggle/input/deepmindmathdata/deepmind_mathemathics.csv'  # replace with your file path
df = pd.read_csv(file_path)

# General parameters of the file
print("First few rows of the file:")
print(df.head())

print("\nSummary statistics of the file:")
print(df.describe())

print("\nInfo about the file:")
print(df.info())

print("\nShape of the file (rows, columns):")
print(df.shape)

# Model setup
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

# Function to evaluate response
def evaluate_response(problem):
    start_time = time.time()
    
    problem = problem + '\nPlease reason step by step, and put your final answer within \\boxed{}.'
    messages = [
        {"role": "user", "content": problem}
    ]
    
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    attention_mask = (input_tensor != tokenizer.pad_token_id).long()
    
    outputs = model.generate(
        input_tensor.to(model.device), 
        attention_mask=attention_mask.to(model.device), 
        max_new_tokens=100
    )

    result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for evaluation: {elapsed_time:.2f} seconds")
    
    return result

# Gradio interface
demo = gr.Interface(
    fn=evaluate_response,
    inputs=[gr.Textbox(label="Question")],
    outputs=gr.Textbox(label="Answer"),
    title="Question and Answer Interface",
    description="Enter a question."
)

if __name__ == "__main__":
    demo.launch(share=True)


CUDA is available. Checking GPU usage...
GPU ID: 0
Name: Tesla T4
Load: 0.0%
Memory Free: 15093.0MB
Memory Used: 3.0MB
Memory Total: 15360.0MB
Temperature: 43.0 °C
UUID: GPU-f975602e-a323-4e35-1875-0c04e4e54bd0

GPU ID: 1
Name: Tesla T4
Load: 0.0%
Memory Free: 15093.0MB
Memory Used: 3.0MB
Memory Total: 15360.0MB
Temperature: 38.0 °C
UUID: GPU-50f61d4f-cca8-44ed-e049-a8ea36fa3796

First few rows of the file:
                                            question  \
0  b'Let w(f) be the second derivative of -18*f +...   
1  b'Suppose 0*l + 3*l - 5*u - 16 = 0, 3*u + 15 =...   
2  b'Determine t so that -10*t - 5*t + 48*t + 5*t...   
3  b'Let c = 3272/5 - 652. Let l(z) = 2*z - 14. L...   
4  b'Let y = 26 - 12. Suppose 8*t + 14 - y = 0. S...   

                            answer  
0                      b'c**5/2\n'  
1                b'2*(i + 1)**3\n'  
2                      b'-13, 0\n'  
3  b'4*v**3*(v + 2)*(3*v + 2)/5\n'  
4                     b'0, 1, 2\n'  

Summary statistics of the fil

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f8122a03bbb0a70da8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/opt/conda/lib/python3.10/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/opt/conda/lib/python3.10/site-packages/gradio/blocks.py", line 1897, in process_api
    result = await self.call_function(
  File "/opt/conda/lib/python3.10/site-packages/gradio/blocks.py", line 1483, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/opt/conda/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "/opt/conda/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 2134, in run_sync_in_worker_thread
    return await future
  File "/opt/conda/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 851, in run

In [2]:
import gradio as gr
import torch
import pandas as pd
import GPUtil
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

# Check GPU availability and usage
if torch.cuda.is_available():
    print("CUDA is available. Checking GPU usage...")
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU ID: {gpu.id}")
        print(f"Name: {gpu.name}")
        print(f"Load: {gpu.load * 100:.1f}%")
        print(f"Memory Free: {gpu.memoryFree:.1f}MB")
        print(f"Memory Used: {gpu.memoryUsed:.1f}MB")
        print(f"Memory Total: {gpu.memoryTotal:.1f}MB")
        print(f"Temperature: {gpu.temperature} °C")
        print(f"UUID: {gpu.uuid}\n")
else:
    print("CUDA is not available. No GPU usage information to display.")

# Load and check the file (update the path as needed)
file_path = '/kaggle/input/deepmindmathdata/deepmind_mathemathics.csv'  # replace with your file path
df = pd.read_csv(file_path)

# General parameters of the file
print("First few rows of the file:")
print(df.head())

print("\nSummary statistics of the file:")
print(df.describe())

print("\nInfo about the file:")
print(df.info())

print("\nShape of the file (rows, columns):")
print(df.shape)

# Model setup
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

# Function to evaluate response
def evaluate_response(problem):
    problem = problem + '\nPlease reason step by step, and put your final answer within \\boxed{}.'
    messages = [
        {"role": "user", "content": problem}
    ]
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=100)

    result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
    return result

# Gradio interface
demo = gr.Interface(
    fn=evaluate_response,
    inputs=[gr.Textbox(label="Question")],
    outputs=gr.Textbox(label="Answer"),
    title="Question and Answer Interface",
    description="Enter a question."
)

if __name__ == "__main__":
    demo.launch()


CUDA is available. Checking GPU usage...
GPU ID: 0
Name: Tesla T4
Load: 0.0%
Memory Free: 15093.0MB
Memory Used: 3.0MB
Memory Total: 15360.0MB
Temperature: 39.0 °C
UUID: GPU-68f79f9b-5d09-5a55-424b-2e46a04da8a5

GPU ID: 1
Name: Tesla T4
Load: 0.0%
Memory Free: 15093.0MB
Memory Used: 3.0MB
Memory Total: 15360.0MB
Temperature: 40.0 °C
UUID: GPU-2ca0a2b6-3f55-0bdb-fb86-de77b4fc93f9

First few rows of the file:
                                            question  \
0  b'Let w(f) be the second derivative of -18*f +...   
1  b'Suppose 0*l + 3*l - 5*u - 16 = 0, 3*u + 15 =...   
2  b'Determine t so that -10*t - 5*t + 48*t + 5*t...   
3  b'Let c = 3272/5 - 652. Let l(z) = 2*z - 14. L...   
4  b'Let y = 26 - 12. Suppose 8*t + 14 - y = 0. S...   

                            answer  
0                      b'c**5/2\n'  
1                b'2*(i + 1)**3\n'  
2                      b'-13, 0\n'  
3  b'4*v**3*(v + 2)*(3*v + 2)/5\n'  
4                     b'0, 1, 2\n'  

Summary statistics of the fil

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://6282f71bc2cc4bdaec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
  File "/opt/conda/lib/python3.10/site-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/opt/conda/lib/python3.10/site-packages/gradio/blocks.py", line 1897, in process_api
    result = await self.call_function(
  File "/opt/conda/lib/python3.10/site-packages/gradio/blocks.py", line 1483, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/opt/conda/lib/python3.10/site-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
  File "

NExt

In [1]:
!pip install gputil pandas transformers psutil

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=70e974a78da5bed586bca7a3317ffba9af956d81837417bbccfeb88b5f04fa25
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil


In [4]:
!pip install --index-url https://download.pytorch.org/whl/nightly/cu118 --pre 'torch==2.2.0.dev20231010+cu118'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://download.pytorch.org/whl/nightly/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 GB 661.6 kB/s eta 0:00:000:0100:05
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 MB 8.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.1.2
    Uninstalling torch-2.1.2:
      Successfully uninstalled torch-2.1.2


In [ ]:
!pip install gputil pandas transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
 
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id
 


In [5]:
messages = [
    {"role": "user", "content": "what is the integral of x^2 from 0 to 2?\nPlease reason step by step, and put your final answer within \\boxed{}."}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
outputs = model.generate(input_tensor.to(model.device), max_new_tokens=100)
 
result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
print(result)

RuntimeError: cutlassF: no kernel found to launch!

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
 
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id
 
messages = [
    {"role": "user", "content": "what is the integral of x^2 from 0 to 2?\nPlease reason step by step, and put your final answer within \\boxed{}."}
]
input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
outputs = model.generate(input_tensor.to(model.device), max_new_tokens=100)
 
result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
print(result)

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


RuntimeError: cutlassF: no kernel found to launch!

In [2]:
import torch
import pandas as pd
import GPUtil
import psutil
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

# Check GPU availability and usage
if torch.cuda.is_available():
    print("CUDA is available. Checking GPU usage...")
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU ID: {gpu.id}")
        print(f"Name: {gpu.name}")
        print(f"Load: {gpu.load * 100:.1f}%")
        print(f"Memory Free: {gpu.memoryFree:.1f}MB")
        print(f"Memory Used: {gpu.memoryUsed:.1f}MB")
        print(f"Memory Total: {gpu.memoryTotal:.1f}MB")
        print(f"Temperature: {gpu.temperature} °C")
        print(f"UUID: {gpu.uuid}\n")
else:
    print("CUDA is not available. Checking CPU usage...")
    # CPU usage information
    print(f"CPU usage: {psutil.cpu_percent()}%")
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Load and check the file (update the path as needed)
file_path = '/kaggle/input/deepmindmathdata/deepmind_mathemathics.csv'  # replace with your file path
df = pd.read_csv(file_path)

# General parameters of the file
print("First few rows of the file:")
print(df.head())

print("\nSummary statistics of the file:")
print(df.describe())

print("\nInfo about the file:")
print(df.info())

print("\nShape of the file (rows, columns):")
print(df.shape)

# Model setup
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

# Function to evaluate response
def evaluate_response(problem):
    start_time = time.time()
    
    problem = problem + '\nPlease reason step by step, and put your final answer within \\boxed{}.'
    messages = [
        {"role": "user", "content": problem}
    ]
    
    input_tensor = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    attention_mask = (input_tensor != tokenizer.pad_token_id).long()
    
    outputs = model.generate(
        input_tensor.to(model.device), 
        attention_mask=attention_mask.to(model.device), 
        max_new_tokens=100
    )

    result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for evaluation: {elapsed_time:.2f} seconds")
    
    return result

if __name__ == "__main__":
    problem = "Enter your math problem here"
    print(evaluate_response(problem))


CUDA is not available. Checking CPU usage...
CPU usage: 25.4%
Memory usage: 4.1%
First few rows of the file:
                                            question  \
0  b'Let w(f) be the second derivative of -18*f +...   
1  b'Suppose 0*l + 3*l - 5*u - 16 = 0, 3*u + 15 =...   
2  b'Determine t so that -10*t - 5*t + 48*t + 5*t...   
3  b'Let c = 3272/5 - 652. Let l(z) = 2*z - 14. L...   
4  b'Let y = 26 - 12. Suppose 8*t + 14 - y = 0. S...   

                            answer  
0                      b'c**5/2\n'  
1                b'2*(i + 1)**3\n'  
2                      b'-13, 0\n'  
3  b'4*v**3*(v + 2)*(3*v + 2)/5\n'  
4                     b'0, 1, 2\n'  

Summary statistics of the file:
                                  question         answer
count                              1999998        1999998
unique                             1999737         701501
top     b'Factor -2/7 + 0*y + 2/7*y**2.\n'  b'-1, 0, 1\n'
freq                                     3          51777

Info abo

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

AttributeError: 'bool' object has no attribute 'long'

In [2]:
import torch
import pandas as pd
import GPUtil
import psutil
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

# Check GPU availability and usage
if torch.cuda.is_available():
    print("CUDA is available. Checking GPU usage...")
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU ID: {gpu.id}")
        print(f"Name: {gpu.name}")
        print(f"Load: {gpu.load * 100:.1f}%")
        print(f"Memory Free: {gpu.memoryFree:.1f}MB")
        print(f"Memory Used: {gpu.memoryUsed:.1f}MB")
        print(f"Memory Total: {gpu.memoryTotal:.1f}MB")
        print(f"Temperature: {gpu.temperature} °C")
        print(f"UUID: {gpu.uuid}\n")
else:
    print("CUDA is not available. Checking CPU usage...")
    # CPU usage information
    print(f"CPU usage: {psutil.cpu_percent()}%")
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Load and check the file (update the path as needed)
file_path = '/kaggle/input/deepmindmathdata/deepmind_mathemathics.csv'  # replace with your file path
df = pd.read_csv(file_path)

# General parameters of the file
print("First few rows of the file:")
print(df.head())

print("\nSummary statistics of the file:")
print(df.describe())

print("\nInfo about the file:")
print(df.info())

print("\nShape of the file (rows, columns):")
print(df.shape)

# Model setup
model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

# Function to evaluate response
def evaluate_response(problem):
    start_time = time.time()
    
    problem = problem + '\nPlease reason step by step, and put your final answer within \\boxed{}.'
    input_tensor = tokenizer.encode(problem, return_tensors="pt")
    attention_mask = (input_tensor != tokenizer.pad_token_id).long()
    
    outputs = model.generate(
        input_tensor.to(model.device), 
        attention_mask=attention_mask.to(model.device), 
        max_new_tokens=100
    )

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken for evaluation: {elapsed_time:.2f} seconds")
    
    return result

if __name__ == "__main__":
    problem = "Enter your math problem here"
    print(evaluate_response(problem))


CUDA is not available. Checking CPU usage...
CPU usage: 28.8%
Memory usage: 4.3%
First few rows of the file:
                                            question  \
0  b'Let w(f) be the second derivative of -18*f +...   
1  b'Suppose 0*l + 3*l - 5*u - 16 = 0, 3*u + 15 =...   
2  b'Determine t so that -10*t - 5*t + 48*t + 5*t...   
3  b'Let c = 3272/5 - 652. Let l(z) = 2*z - 14. L...   
4  b'Let y = 26 - 12. Suppose 8*t + 14 - y = 0. S...   

                            answer  
0                      b'c**5/2\n'  
1                b'2*(i + 1)**3\n'  
2                      b'-13, 0\n'  
3  b'4*v**3*(v + 2)*(3*v + 2)/5\n'  
4                     b'0, 1, 2\n'  

Summary statistics of the file:
                                  question         answer
count                              1999998        1999998
unique                             1999737         701501
top     b'Factor -2/7 + 0*y + 2/7*y**2.\n'  b'-1, 0, 1\n'
freq                                     3          51777

Info abo

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

AttributeError: 'bool' object has no attribute 'long'

In [1]:
!pip install gputil pandas transformers psutil

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=1459b7cf081fb0f22e923f5c96bc9cff3b538db4b3c2dfc33e110aed88a66c7f
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil


In [2]:
import torch
import pandas as pd
import GPUtil
import psutil
import time
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

# Check GPU availability and usage
if torch.cuda.is_available():
    print("CUDA is available. Checking GPU usage...")
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU ID: {gpu.id}")
        print(f"Name: {gpu.name}")
        print(f"Load: {gpu.load * 100:.1f}%")
        print(f"Memory Free: {gpu.memoryFree:.1f}MB")
        print(f"Memory Used: {gpu.memoryUsed:.1f}MB")
        print(f"Memory Total: {gpu.memoryTotal:.1f}MB")
        print(f"Temperature: {gpu.temperature} °C")
        print(f"UUID: {gpu.uuid}\n")
else:
    print("CUDA is not available. Checking CPU usage...")
    # CPU usage information
    print(f"CPU usage: {psutil.cpu_percent()}%")
    print(f"Memory usage: {psutil.virtual_memory().percent}%")

# Load and check the file (update the path as needed)
file_path = '/kaggle/input/deepmindmathdata/deepmind_mathemathics.csv'  # replace with your file path
df = pd.read_csv(file_path)

# General parameters of the file
print("First few rows of the file:")
print(df.head())

print("\nSummary statistics of the file:")
print(df.describe())

print("\nInfo about the file:")
print(df.info())

print("\nShape of the file (rows, columns):")
print(df.shape)

model_name = "deepseek-ai/deepseek-math-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

def evaluate_response(problem):
    problem = problem + '\nPlease reason step by step, and put your final answer within \\boxed{}.'
    input_tensor = tokenizer(problem, return_tensors="pt")
    outputs = model.generate(input_tensor.to(model.device), max_new_tokens=100)

    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result

if __name__ == "__main__":
    problem = "Enter your math problem here"
    print(evaluate_response(problem))


CUDA is not available. Checking CPU usage...
CPU usage: 7.8%
Memory usage: 4.9%
First few rows of the file:
                                            question  \
0  b'Let w(f) be the second derivative of -18*f +...   
1  b'Suppose 0*l + 3*l - 5*u - 16 = 0, 3*u + 15 =...   
2  b'Determine t so that -10*t - 5*t + 48*t + 5*t...   
3  b'Let c = 3272/5 - 652. Let l(z) = 2*z - 14. L...   
4  b'Let y = 26 - 12. Suppose 8*t + 14 - y = 0. S...   

                            answer  
0                      b'c**5/2\n'  
1                b'2*(i + 1)**3\n'  
2                      b'-13, 0\n'  
3  b'4*v**3*(v + 2)*(3*v + 2)/5\n'  
4                     b'0, 1, 2\n'  

Summary statistics of the file:
                                  question         answer
count                              1999998        1999998
unique                             1999737         701501
top     b'Factor -2/7 + 0*y + 2/7*y**2.\n'  b'-1, 0, 1\n'
freq                                     3          51777

Info abou

tokenizer_config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

AttributeError: 